# Classification Prompt Experimentation

## Setup

In [1]:
!ls

README.md		    miniconda.sh  quick_start_pytorch.ipynb   wandb
eliciting-latent-sentiment  miniconda3	  quick_start_pytorch_images


In [2]:
%cd eliciting-latent-sentiment

/notebooks/eliciting-latent-sentiment


In [3]:
#!source activate circuits/bin/activate

In [4]:
#!pip install git+https://github.com/neelnanda-io/TransformerLens.git
!pip install git+https://github.com/glerzing/TransformerLens.git@stable_lm
!pip install circuitsvis
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git

  Cloning https://github.com/glerzing/TransformerLens.git (to revision stable_lm) to /tmp/pip-req-build-plhy9oit
  Running command git clone --filter=blob:none --quiet https://github.com/glerzing/TransformerLens.git /tmp/pip-req-build-plhy9oit
  Running command git checkout -b stable_lm --track origin/stable_lm
  Switched to a new branch 'stable_lm'
  Branch 'stable_lm' set up to track remote branch 'stable_lm' from 'origin'.
  Resolved https://github.com/glerzing/TransformerLens.git to commit 049f56f810292ad05be77633898472c700ab8e27
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 84.8 MB/s et

In [5]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [6]:
import os
import pathlib
from typing import List, Optional, Union

import torch
import numpy as np
import yaml
import pickle
import einops
from fancy_einsum import einsum


import circuitsvis as cv

import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
import transformer_lens.patching as patching

from torch import Tensor
from tqdm.notebook import tqdm
from jaxtyping import Float, Int, Bool
from typing import List, Optional, Callable, Tuple, Dict, Literal, Set
from rich import print as rprint

from typing import List, Union
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

from functools import partial

from torchtyping import TensorType as TT

from path_patching import Node, IterNode, path_patch, act_patch
from neel_plotly import imshow as imshow_n

from utils.visualization import get_attn_head_patterns, imshow_p, plot_attention_heads, scatter_attention_and_contribution_simple

from utils.prompts import get_dataset
from utils.circuit_analysis import get_logit_diff, logit_diff_denoising, logit_diff_noising

In [7]:
import torch
torch.set_grad_enabled(False)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
#import plotly
#plotly.offline.init_notebook_mode()

In [9]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[utils.to_numpy(tensor1), utils.to_numpy(tensor2)], **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Exploratory Analysis


### Transformers Version

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

tokenizer = AutoTokenizer.from_pretrained("StabilityAI/stablelm-tuned-alpha-7b")
model = AutoModelForCausalLM.from_pretrained("StabilityAI/stablelm-tuned-alpha-7b", cache_dir="model_cache")
model.half().cuda()

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

prompt = f"{system_prompt}<|USER|>What's your mood today?<|ASSISTANT|>"
prompt = f"<|USER|>What's your mood today?<|ASSISTANT|>"
prompt =   (f"{system_prompt}<|USER|>Jim's Review: 'I thought this movie was spectacular, I loved it. The acting was great, the plot was beautiful, and overall the movie was just very good.'" + "\n" +
            "Ann's Review: 'I thought this movie was horrific, I hated it. The acting was terrible, the plot was disgusting, and overall the movie was just very bad.'"+ "\n" +
            "What is the sentiment of Jim's review? Answer with 'Negative' or 'Positive'\n<|ASSISTANT|>")

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=64,
  temperature=0.7,
  do_sample=True,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()])
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Jim's Review: 'I thought this movie was spectacular, I loved it. The acting was great, the plot was beautiful, and overall the movie was just very good.'
Ann's Review: 'I thought this movie was horrific, I hated it. The acting was terrible, the plot was disgusting, and overall the movie was just very bad.'
What is the sentiment of Jim's review? Answer with 'Negative' or 'Positive'
Negative.


In [35]:
tokenizer = AutoTokenizer.from_pretrained("StabilityAI/stablelm-tuned-alpha-7b")
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [38]:
[tokenizer.decode(x) for x in inputs["input_ids"][0]]

['<|USER|>',
 'Kate',
 ' lost',
 ' her',
 ' house',
 ' in',
 ' a',
 ' fire',
 ' and',
 ' felt',
 ' miserable',
 '.',
 ' Jim',
 ' won',
 ' many',
 ' millions',
 ' in',
 ' the',
 ' lottery',
 ' and',
 ' felt',
 ' excited',
 '.',
 ' What',
 ' is',
 ' Kate',
 "'s",
 ' mood',
 '?',
 '\n',
 '<|ASSISTANT|>']

In [37]:
inputs

{'input_ids': tensor([[50278, 47085,  3663,   617,  2419,   275,   247,  3289,   285,  3543,
         28714,    15,  8438,  1912,  1142,  9790,   275,   253, 36284,   285,
          3543,  9049,    15,  1737,   310, 20428,   434, 12315,    32,   187,
         50279]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [11]:
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50688, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
       

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
# an example prompt
prompt =   ("Review A: 'I thought this movie was spectacular, I loved it. The acting was great, the plot was beautiful, and overall the movie was just very good.'" + "\n" +
            "Review B: 'I thought this movie was horrific, I hated it. The acting was terrible, the plot was disgusting, and overall the movie was just very bad.'"+ "\n" +
            "Did Reviewer B like the movie?")

# encode the prompt and pass it through the model
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# generate text
output = model.generate(input_ids, max_new_tokens=3, temperature=0.9, do_sample=True)

# decode the output
generated_text = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 If yes,


### TransformerLens Version

In [10]:
from transformers import AutoModelForCausalLM

def load_model(model_name):
    if model_name == "EleutherAI/pythia-6.9b" or model_name == "StabilityAI/stablelm-tuned-alpha-7b":
        source_model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="model_cache").to('cpu').bfloat16()
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = HookedTransformer.from_pretrained(
            model_name,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False,
            tokenizer=tokenizer,
            hf_model=source_model,
        )
    else:
        model = HookedTransformer.from_pretrained(
            model_name,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False
        )
    return model


In [11]:
model = load_model("EleutherAI/pythia-2.8b")
model.set_use_hook_mlp_in(True)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer


### Initial Examination

In [36]:
example_prompt =    ("Anne hates parties. John loves parties. One day they are invited to a grand gala. Anne feels very")

print(example_prompt)
example_answer = " happy"

res = utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, top_k=10)

Anne hates parties. John loves parties. One day they are invited to a grand gala. Anne feels very
Tokenized prompt: ['<|endoftext|>', 'Anne', ' hates', ' parties', '.', ' John', ' loves', ' parties', '.', ' One', ' day', ' they', ' are', ' invited', ' to', ' a', ' grand', ' gal', 'a', '.', ' Anne', ' feels', ' very']
Tokenized answer: [' happy']


Performance on answer token:
Rank: 31       Logit: 12.97 Prob:  0.41% Token: | happy|

Top 0th token. Logit: 16.57 Prob: 15.00% Token: | out|
Top 1th token. Logit: 16.46 Prob: 13.50% Token: | uncomfortable|
Top 2th token. Logit: 15.90 Prob:  7.68% Token: | awkward|
Top 3th token. Logit: 15.64 Prob:  5.92% Token: | nervous|
Top 4th token. Logit: 15.07 Prob:  3.35% Token: | self|
Top 5th token. Logit: 15.01 Prob:  3.17% Token: | left|
Top 6th token. Logit: 14.94 Prob:  2.94% Token: | much|
Top 7th token. Logit: 14.81 Prob:  2.60% Token: | strange|
Top 8th token. Logit: 14.78 Prob:  2.52% Token: | shy|
Top 9th token. Logit: 14.56 Prob:  2.01% Token: | un|


Ranks of the answer tokens: [(' happy', 31)]

In [14]:
example_prompt =    ("Joe, the excited puppy, and Fred, the lazy cat, discovered a new toy. Joe wagged his tail and jumped around the toy with joy. Fred, however, looked at the toy and yawned, unimpressed. They spent the day with Joe playing energetically and Fred snoozing beside him. At the end of the day, Joe felt")
print(example_prompt)
example_answer = " happy"

utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, top_k=15)

Joe, the excited puppy, and Fred, the lazy cat, discovered a new toy. Joe wagged his tail and jumped around the toy with joy. Fred, however, looked at the toy and yawned, unimpressed. They spent the day with Joe playing energetically and Fred snoozing beside him. At the end of the day, Joe felt
Tokenized prompt: ['<|endoftext|>', 'Joe', ',', ' the', ' excited', ' puppy', ',', ' and', ' Fred', ',', ' the', ' lazy', ' cat', ',', ' discovered', ' a', ' new', ' toy', '.', ' Joe', ' w', 'agged', ' his', ' tail', ' and', ' jumped', ' around', ' the', ' toy', ' with', ' joy', '.', ' Fred', ',', ' however', ',', ' looked', ' at', ' the', ' toy', ' and', ' ya', 'wn', 'ed', ',', ' un', 'imp', 'ressed', '.', ' They', ' spent', ' the', ' day', ' with', ' Joe', ' playing', ' energet', 'ically', ' and', ' Fred', ' s', 'no', 'oz', 'ing', ' beside', ' him', '.', ' At', ' the', ' end', ' of', ' the', ' day', ',', ' Joe', ' felt']
Tokenized answer: [' happy']


Performance on answer token:
Rank: 13       Logit: 14.08 Prob:  1.44% Token: | happy|

Top 0th token. Logit: 16.22 Prob: 12.25% Token: | a|
Top 1th token. Logit: 15.69 Prob:  7.19% Token: | so|
Top 2th token. Logit: 15.62 Prob:  6.72% Token: | that|
Top 3th token. Logit: 15.47 Prob:  5.78% Token: | exhausted|
Top 4th token. Logit: 15.29 Prob:  4.83% Token: | he|
Top 5th token. Logit: 15.13 Prob:  4.10% Token: | the|
Top 6th token. Logit: 15.11 Prob:  4.05% Token: | tired|
Top 7th token. Logit: 15.04 Prob:  3.77% Token: | very|
Top 8th token. Logit: 14.91 Prob:  3.30% Token: | like|
Top 9th token. Logit: 14.54 Prob:  2.28% Token: | guilty|
Top 10th token. Logit: 14.26 Prob:  1.72% Token: | his|
Top 11th token. Logit: 14.25 Prob:  1.70% Token: | good|
Top 12th token. Logit: 14.09 Prob:  1.46% Token: | satisfied|
Top 13th token. Logit: 14.08 Prob:  1.44% Token: | happy|
Top 14th token. Logit: 14.07 Prob:  1.43% Token: | sad|


Ranks of the answer tokens: [(' happy', 13)]

In [25]:
# Define the list of stories
stories = [
    {
        "original": "Claire, the social butterfly, flourishes in crowded, lively parties. Dave, the reclusive writer, cherishes solitude and silence. Out of the blue, they received invitations to a grand gala.",
        "reversed": "Claire, the social butterfly, flourishes in crowded, lively parties. Dave, the reclusive writer, cherishes solitude and silence. One day, they were offered an opportunity to join a peaceful, secluded writer's retreat in the woods.",
        "characters": ["Claire", "Dave"]
    },
    {
        "original": "Ivy, the thrill-seeking skydiver, adores high-speed and adrenaline rushes. Jake, the cautious librarian, prefers safety and tranquility. One day, they were offered a chance to jump from an airplane.",
        "reversed": "Ivy, the thrill-seeking skydiver, adores high-speed and adrenaline rushes. Jake, the cautious librarian, prefers safety and tranquility. Suddenly, they were placed in a quiet, organized library full of rare books.",
        "characters": ["Ivy", "Jake"]
    },
    {
        "original": "Liam, the daredevil stuntman, gets a thrill out of dangerous action. Mia, the careful nurse, values safety and caution. Suddenly, they were offered an opportunity to join a high-adrenaline adventure camp.",
        "reversed": "Liam, the daredevil stuntman, gets a thrill out of dangerous action. Mia, the careful nurse, values safety and caution. One day, they were tasked to work in a safe, calm recovery ward.",
        "characters": ["Liam", "Mia"]
    },
    {
        "original": "Xander, the energetic DJ, loves loud music and vibrant dance floors. Yara, the contemplative librarian, cherishes silence and tranquility. One day, their friends asked them to go to a lively, crowded music festival together.",
        "reversed": "Xander, the energetic DJ, loves loud music and vibrant dance floors. Yara, the contemplative librarian, cherishes silence and tranquility. One day, they were placed in a quiet, ancient library.",
        "characters": ["Xander", "Yara"]
    },
]

stories_2 = [
    {
        "original": "Claire, the social butterfly, flourishes in crowded, lively parties. Dave, the reclusive writer, cherishes solitude and silence. Out of the blue, they received invitations to a grand gala.",
        "reversed": "Claire, the social butterfly, flourishes in crowded, lively parties. Dave, the reclusive writer, cherishes solitude and silence. Unexpectedly, they were booked on a solitary retreat on a remote, desolate island.",
        "characters": ["Claire", "Dave"]
    },
    {
        "original": "Ivy, the thrill-seeking skydiver, adores high-speed and adrenaline rushes. Jake, the cautious librarian, prefers safety and tranquility. One day, they were offered a chance to jump from an airplane.",
        "reversed": "Ivy, the thrill-seeking skydiver, adores high-speed and adrenaline rushes. Jake, the cautious librarian, prefers safety and tranquility. Suddenly, they were trapped in a silent, dimly-lit room with nothing but a pile of dusty old books.",
        "characters": ["Ivy", "Jake"]
    },
    {
        "original": "Liam, the daredevil stuntman, gets a thrill out of dangerous action. Mia, the careful nurse, values safety and caution. Suddenly, they were offered roles in a high-risk action movie.",
        "reversed": "Liam, the daredevil stuntman, gets a thrill out of dangerous action. Mia, the careful nurse, values safety and caution. One day, they found themselves in an extremely safe and slow-paced senior care facility.",
        "characters": ["Liam", "Mia"]
    },
    {
        "original": "Xander, the energetic DJ, loves loud music and vibrant dance floors. Yara, the contemplative librarian, cherishes silence and tranquility. Suddenly, they were asked to perform at a bustling music festival.",
        "reversed": "Xander, the energetic DJ, loves loud music and vibrant dance floors. Yara, the contemplative librarian, cherishes silence and tranquility. Out of the blue, they were sent to a secluded, noise-free monastery in the hills.",
        "characters": ["Xander", "Yara"]
    },
    {
        "original": "Oliver, the intrepid explorer, relishes the thrill of uncharted territories. Ava, the meticulous scientist, values control and predictability. Out of nowhere, they were teleported to an alien planet.",
        "reversed": "Oliver, the intrepid explorer, relishes the thrill of uncharted territories. Ava, the meticulous scientist, values control and predictability. One day, they were confined to a highly regulated, sterile laboratory.",
        "characters": ["Oliver", "Ava"]
    },
    {
        "original": "Sophie, the social media influencer, thrives on spotlight and popularity. Ryan, the privacy-loving introvert, appreciates anonymity and peace. Suddenly, they were cast in a reality TV show.",
        "reversed": "Sophie, the social media influencer, thrives on spotlight and popularity. Ryan, the privacy-loving introvert, appreciates anonymity and peace. One day, they found themselves living off-grid in a remote mountain cabin with no internet.",
        "characters": ["Sophie", "Ryan"]
    },
    {
        "original": "Mason, the athletic football player, loves competition and physical challenge. Isabella, the intellectual philosopher, relishes deep thoughts and cerebral challenges. Out of the blue, they were invited to a state-level football tournament.",
        "reversed": "Mason, the athletic football player, loves competition and physical challenge. Isabella, the intellectual philosopher, relishes deep thoughts and cerebral challenges. One day, they were enrolled in a high-level philosophical debate.",
        "characters": ["Mason", "Isabella"]
    },
    {
        "original": "Ella, the daredevil bungee jumper, thrives on heights and speed. Benjamin, the grounded cook, appreciates stability and calmness. One day, they were offered to bungee jump from the tallest bridge.",
        "reversed": "Ella, the daredevil bungee jumper, thrives on heights and speed. Benjamin, the grounded cook, appreciates stability and calmness. Suddenly, they found themselves in a calm and slow-paced cooking class.",
        "characters": ["Ella", "Benjamin"]
    },
    {
        "original": "Amelia, the enthusiastic extrovert, loves crowds and loud parties. Lucas, the shy programmer, prefers solitude and silence. Out of nowhere, they were invited to a grand music concert.",
        "reversed": "Amelia, the enthusiastic extrovert, loves crowds and loud parties. Lucas, the shy programmer, prefers solitude and silence. One day, they were assigned to a solitary coding project that required complete silence.",
        "characters": ["Amelia", "Lucas"]
    },
]



story_counter = 0
# Iterate through each story
for story in stories_2:
    print("======================================================================================================")
    print(f"Story {story_counter}")
    print("======================================================================================================\n\n")
    
    print(f"Original Story:")
    print("======================================================================================================")
    print(f"{story['original']}\n")
    for character in story["characters"]:
        # Print the character's name
        print("---------------------------------------------------------")
        print(f"Character: {character}")
        print("---------------------------------------------------------")
        # Get the end of the original story
        original_end = story["original"] + character.capitalize() + " felt very"
        # Get the LLM prediction for the original story
        print(f"Mood Prediction:")
        original_prediction = utils.test_prompt(original_end, "", model, prepend_bos=True, top_k=5)
    
    print(f"Reversed Story:")
    print("======================================================================================================")
    print(f"{story['reversed']}\n")
    for character in story["characters"]:
        print("---------------------------------------------------------")
        print(f"Character: {character}")
        print("---------------------------------------------------------")
        print(f"Mood Prediction:")
        reversed_end = story["reversed"] + character.capitalize() + " felt very"
        # Get the LLM prediction for the reversed story
        reversed_prediction = utils.test_prompt(reversed_end, "", model, prepend_bos=True, top_k=5)
    story_counter += 1


Story 0


Original Story:
Claire, the social butterfly, flourishes in crowded, lively parties. Dave, the reclusive writer, cherishes solitude and silence. Out of the blue, they received invitations to a grand gala.

---------------------------------------------------------
Character: Claire
---------------------------------------------------------
Mood Prediction:
Tokenized prompt: ['<|endoftext|>', 'Cl', 'aire', ',', ' the', ' social', ' butterfly', ',', ' flour', 'ishes', ' in', ' crowded', ',', ' lively', ' parties', '.', ' Dave', ',', ' the', ' re', 'clusive', ' writer', ',', ' cher', 'ishes', ' sol', 'itude', ' and', ' silence', '.', ' Out', ' of', ' the', ' blue', ',', ' they', ' received', ' inv', 'itations', ' to', ' a', ' grand', ' gal', 'a', '.', 'Cl', 'aire', ' felt', ' very']
Tokenized answer: [' ']


Performance on answer token:
Rank: 578      Logit:  8.52 Prob:  0.00% Token: | |

Top 0th token. Logit: 17.10 Prob: 23.38% Token: | excited|
Top 1th token. Logit: 15.83 Prob:  6.55% Token: | much|
Top 2th token. Logit: 15.76 Prob:  6.11% Token: | nervous|
Top 3th token. Logit: 15.44 Prob:  4.43% Token: | special|
Top 4th token. Logit: 15.34 Prob:  4.01% Token: | out|


Ranks of the answer tokens: [(' ', 578)]

---------------------------------------------------------
Character: Dave
---------------------------------------------------------
Mood Prediction:
Tokenized prompt: ['<|endoftext|>', 'Cl', 'aire', ',', ' the', ' social', ' butterfly', ',', ' flour', 'ishes', ' in', ' crowded', ',', ' lively', ' parties', '.', ' Dave', ',', ' the', ' re', 'clusive', ' writer', ',', ' cher', 'ishes', ' sol', 'itude', ' and', ' silence', '.', ' Out', ' of', ' the', ' blue', ',', ' they', ' received', ' inv', 'itations', ' to', ' a', ' grand', ' gal', 'a', '.', 'Dave', ' felt', ' very']
Tokenized answer: [' ']


...

Ranks of the answer tokens: [(' ', 414)]

---------------------------------------------------------
Character: Lucas
---------------------------------------------------------
Mood Prediction:
Tokenized prompt: ['<|endoftext|>', 'Am', 'elia', ',', ' the', ' enthusiastic', ' ext', 'ro', 'vert', ',', ' loves', ' crowds', ' and', ' loud', ' parties', '.', ' Lucas', ',', ' the', ' shy', ' programmer', ',', ' prefers', ' sol', 'itude', ' and', ' silence', '.', ' One', ' day', ',', ' they', ' were', ' assigned', ' to', ' a', ' solitary', ' coding', ' project', ' that', ' required', ' complete', ' silence', '.', 'Luc', 'as', ' felt', ' very']
Tokenized answer: [' ']


Performance on answer token:
Rank: 312      Logit: 10.10 Prob:  0.01% Token: | |

Top 0th token. Logit: 17.50 Prob: 16.65% Token: | uncomfortable|
Top 1th token. Logit: 16.85 Prob:  8.70% Token: | nervous|
Top 2th token. Logit: 16.74 Prob:  7.76% Token: | anxious|
Top 3th token. Logit: 16.59 Prob:  6.72% Token: | lonely|
Top 4th token. Logit: 16.48 Prob:  6.01% Token: | alone|


Ranks of the answer tokens: [(' ', 312)]

In [37]:
def print_lists(list1, list2):
    # Find the maximum length
    max_len = max(len(list1), len(list2))

    # Print the items and their indices
    for i in range(max_len):
        item1 = list1[i] if i < len(list1) else None
        item2 = list2[i] if i < len(list2) else None
        print(f"Index: {i}, '{item1}', '{item2}'")


In [40]:
story = {
        "original": "Penny, the puppy, and Whiskers, the cat, discovered a bouncing ball. Penny was thrilled and started chasing the ball. Whiskers, however, was startled by the ball's movement and climbed a tree. Penny had a fun-filled day playing, while Whiskers watched warily from above.",
        "reversed": "Penny, the puppy, and Whiskers, the cat, found a quiet corner. Penny was bored without anything to play with, but Whiskers curled up and started purring. Penny wandered around restlessly, while Whiskers had a peaceful nap.",
        "characters": ["Penny", "Whiskers"]
    }
print(model.to_str_tokens(story["original"]))
print(model.to_str_tokens(story["reversed"]))
print_lists(model.to_str_tokens(story["original"]), model.to_str_tokens(story["reversed"]))

for character in story["characters"]:
    # Print the character's name
    print("---------------------------------------------------------")
    print(f"Character: {character}")
    print("---------------------------------------------------------")
    # Get the end of the original story
    original_end = story["original"] + character.capitalize() + " felt very"
    # Get the LLM prediction for the original story
    print(f"Mood Prediction:")
    original_prediction = utils.test_prompt(original_end, "", model, prepend_bos=True, top_k=5)
    
print(f"Reversed Story:")
print("======================================================================================================")
print(f"{story['reversed']}\n")
for character in story["characters"]:
    print("---------------------------------------------------------")
    print(f"Character: {character}")
    print("---------------------------------------------------------")
    print(f"Mood Prediction:")
    reversed_end = story["reversed"] + character.capitalize() + " felt very"
    # Get the LLM prediction for the reversed story
    reversed_prediction = utils.test_prompt(reversed_end, "", model, prepend_bos=True, top_k=5)

['<|endoftext|>', 'P', 'enny', ',', ' the', ' puppy', ',', ' and', ' Wh', 'isk', 'ers', ',', ' the', ' cat', ',', ' discovered', ' a', ' bouncing', ' ball', '.', ' Penny', ' was', ' thrilled', ' and', ' started', ' chasing', ' the', ' ball', '.', ' Wh', 'isk', 'ers', ',', ' however', ',', ' was', ' startled', ' by', ' the', ' ball', "'s", ' movement', ' and', ' climbed', ' a', ' tree', '.', ' Penny', ' had', ' a', ' fun', '-', 'filled', ' day', ' playing', ',', ' while', ' Wh', 'isk', 'ers', ' watched', ' war', 'ily', ' from', ' above', '.']
['<|endoftext|>', 'P', 'enny', ',', ' the', ' puppy', ',', ' and', ' Wh', 'isk', 'ers', ',', ' the', ' cat', ',', ' found', ' a', ' quiet', ' corner', '.', ' Penny', ' was', ' bored', ' without', ' anything', ' to', ' play', ' with', ',', ' but', ' Wh', 'isk', 'ers', ' curled', ' up', ' and', ' started', ' pur', 'ring', '.', ' Penny', ' wandered', ' around', ' rest', 'lessly', ',', ' while', ' Wh', 'isk', 'ers', ' had', ' a', ' peaceful', ' nap', '

Performance on answer token:
Rank: 637      Logit:  8.62 Prob:  0.00% Token: | |

Top 0th token. Logit: 18.24 Prob: 21.66% Token: | proud|
Top 1th token. Logit: 17.67 Prob: 12.23% Token: | happy|
Top 2th token. Logit: 16.98 Prob:  6.16% Token: | sad|
Top 3th token. Logit: 16.98 Prob:  6.13% Token: | sorry|
Top 4th token. Logit: 16.93 Prob:  5.82% Token: | good|


Ranks of the answer tokens: [(' ', 637)]

---------------------------------------------------------
Character: Whiskers
---------------------------------------------------------
Mood Prediction:
Tokenized prompt: ['<|endoftext|>', 'P', 'enny', ',', ' the', ' puppy', ',', ' and', ' Wh', 'isk', 'ers', ',', ' the', ' cat', ',', ' discovered', ' a', ' bouncing', ' ball', '.', ' Penny', ' was', ' thrilled', ' and', ' started', ' chasing', ' the', ' ball', '.', ' Wh', 'isk', 'ers', ',', ' however', ',', ' was', ' startled', ' by', ' the', ' ball', "'s", ' movement', ' and', ' climbed', ' a', ' tree', '.', ' Penny', ' had', ' a', ' fun', '-', 'filled', ' day', ' playing', ',', ' while', ' Wh', 'isk', 'ers', ' watched', ' war', 'ily', ' from', ' above', '.', 'Wh', 'isk', 'ers', ' felt', ' very']
Tokenized answer: [' ']


...

Ranks of the answer tokens: [(' ', 536)]

---------------------------------------------------------
Character: Whiskers
---------------------------------------------------------
Mood Prediction:
Tokenized prompt: ['<|endoftext|>', 'P', 'enny', ',', ' the', ' puppy', ',', ' and', ' Wh', 'isk', 'ers', ',', ' the', ' cat', ',', ' found', ' a', ' quiet', ' corner', '.', ' Penny', ' was', ' bored', ' without', ' anything', ' to', ' play', ' with', ',', ' but', ' Wh', 'isk', 'ers', ' curled', ' up', ' and', ' started', ' pur', 'ring', '.', ' Penny', ' wandered', ' around', ' rest', 'lessly', ',', ' while', ' Wh', 'isk', 'ers', ' had', ' a', ' peaceful', ' nap', '.', 'Wh', 'isk', 'ers', ' felt', ' very']
Tokenized answer: [' ']


Performance on answer token:
Rank: 499      Logit:  9.09 Prob:  0.00% Token: | |

Top 0th token. Logit: 17.19 Prob: 10.44% Token: | comfortable|
Top 1th token. Logit: 16.97 Prob:  8.42% Token: | happy|
Top 2th token. Logit: 16.88 Prob:  7.68% Token: | safe|
Top 3th token. Logit: 16.54 Prob:  5.49% Token: | content|
Top 4th token. Logit: 16.45 Prob:  4.97% Token: | warm|


Ranks of the answer tokens: [(' ', 499)]

In [40]:
# Append the example prompt to a markdown file for later use, with a version number based on the previous entry
# This is useful for keeping track of the prompts used in each experiment
with open("prompt_log.md", "a") as f:
    # count the number of lines in the file divided by 4 (each prompt is 4 lines)
    # this is the version number of the prompt
    version = int(os.popen("wc -l prompt_log.md | awk '{print $1}'").read()) // 4
    f.write(f"## Version {version}\n")
    f.write(f"### Prompt Text: {example_prompt}\n")
    f.write(f"### Answer: {example_answer}\n\n")

f.close()


### Dataset Construction

In [12]:
pos_answers = [" Positive", " positive"] #, " amazing", " good"]
neg_answers = [" Negative", " negative"] #, " terrible", " bad"]
all_prompts, answer_tokens, clean_tokens, corrupted_tokens = get_dataset(
    model, device, 2, "multi_subject_1", pos_answers, neg_answers
)

Using Pythia model
Count of words: 20
Using Pythia model
Count of words: 15
Using Pythia model
Count of words: 19
Using Pythia model
Count of words: 2
Using Pythia model
Count of words: 2
Using Pythia model
Count of words: 7


In [13]:
len(all_prompts), answer_tokens.shape, clean_tokens.shape, corrupted_tokens.shape

(72, torch.Size([72, 2, 2]), torch.Size([72, 76]), torch.Size([72, 76]))

In [14]:
for i in range(0, len(all_prompts), 1):
    logits, _ = model.run_with_cache(all_prompts[i])
    log_diff = get_logit_diff(logits, answer_tokens[i].unsqueeze(0))
    #if log_diff < 0.1:
    print(all_prompts[i])
    print(model.to_str_tokens(answer_tokens[i][0]))
    print(log_diff, "\n")

Review A: 'I thought this movie was perfect, I loved it. The acting was fantastic, the plot was good, and overall the movie was just very good.'
Review B: 'I thought this movie was dreadful, I hated it. The acting was bad, the plot was miserable, and overall the movie was just very bad.'
Review A Sentiment:
[' Positive', ' Negative']
tensor(1.2028, device='cuda:0') 

Review A: 'I thought this movie was perfect, I loved it. The acting was fantastic, the plot was good, and overall the movie was just very good.'
Review B: 'I thought this movie was dreadful, I hated it. The acting was bad, the plot was miserable, and overall the movie was just very bad.'
Review B Sentiment:
[' Negative', ' Positive']
tensor(0.1087, device='cuda:0') 

Review A: 'I thought this movie was dreadful, I hated it. The acting was bad, the plot was miserable, and overall the movie was just very bad.'
Review B: 'I thought this movie was perfect, I loved it. The acting was fantastic, the plot was good, and overall th

In [32]:
all_prompts = all_prompts[:36]
answer_tokens = answer_tokens[:36]
clean_tokens = clean_tokens[:36]
corrupted_tokens = corrupted_tokens[:36]

#### Logit Differences

In [35]:
pos_logits, pos_cache = model.run_with_cache(clean_tokens[0::2,:])
pos_logit_diff = get_logit_diff(pos_logits, answer_tokens[0::2,:])
pos_logit_diff

tensor(-0.9954, device='cuda:0')

In [34]:
neg_logits, neg_cache = model.run_with_cache(clean_tokens[1::2,:])
neg_logit_diff = get_logit_diff(neg_logits, answer_tokens[1::2,:])
neg_logit_diff

tensor(0.9937, device='cuda:0')

In [107]:
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = get_logit_diff(clean_logits, answer_tokens, per_prompt=False)
clean_logit_diff

tensor(0.8580, device='cuda:0')

In [108]:
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_tokens, per_prompt=False)
corrupted_logit_diff

tensor(-0.8002, device='cuda:0')

In [82]:
imshow_p(
    results['z'],
    title="Direct effect on logit diff (patch from head output -> final resid)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

In [85]:
plot_attention_heads(-results['z'].cuda(), top_n=15, range_x=[0, 0.7])

Total logit diff contribution above threshold: 1.05
